In [ ]:
import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader

sys.path.append(os.path.join(sys.path[0], '../src'))
import data
import utils
import model
from model import EncoderRNN
from model import DecoderRNN

%load_ext autoreload
%autoreload 2

## Load and process training data

In [ ]:
# Change to directory containing English to Tamil transliteration data.
DATA_DIR = "../data/aksharantar_sampled/tam/"
os.chdir(DATA_DIR)
os.listdir()

The class WordDataset reads and stores words in onehot representation.

In [ ]:
# dataset = data.WordDataset() # By default reads 'tam_train.csv'.
# with open('dataset.pkl', 'wb') as file:
    # pickle.dump(dataset, file)

In [ ]:
with open('dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

The above object `dataset[i]` returns i-th data point: a word-pair in one-hot represenation. 

In [ ]:
x, y = dataset[0]
print(x.shape)
print(y.shape)

In [ ]:
dataset.decode_eng_word(x)

In [ ]:
dataset.decode_tam_word(y)

### Sampling data

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
next(iter(train_dataloader))

## Test Encoder

In [ ]:
from model import EncoderRNN
encoder = EncoderRNN(cell_type='gru')
output, hidden = encoder.forward(x)

In [ ]:
out = torch.sum(output)
out.backward()

In [ ]:
for param in encoder.parameters():
    print(param.grad)

In [ ]:
print(output.shape)
print(hidden.shape)

### Decoder

In [ ]:
y[0].shape

In [ ]:
import model
from model import DecoderRNN

decoder = DecoderRNN(cell_type='gru')
out, hidden = decoder.forward(y[0], hidden)

In [ ]:
hidden.shape

In [ ]:
print(out.shape)
print(hidden.shape)

## Loss

In [ ]:
target = torch.zeros(128)
target[0] = 1
target = torch.tensor([torch.where(target == 1)[0]])
y_hat = out
criterion = nn.NLLLoss()
loss = criterion(y_hat, target)

In [ ]:
for param in encoder.parameters():
    print(param.grad)
    break

## Test Training

Code prior to calling `train`

In [ ]:
encoder = EncoderRNN(cell_type='gru')
decoder = DecoderRNN(cell_type='gru')
encoder, decoder, loss_seq = utils.train(dataset, encoder, decoder, 0.01, 2000)

In [ ]:
plt.plot(loss_seq)

## Evaluation

In [ ]:
x, y = next(iter(dataset))

In [ ]:
encoder.initialize_hidden()

In [ ]:
y_pred = utils.predict(x, encoder, decoder)

In [ ]:
torch.sum(y_pred)
dataset.decode_tam_word(y_pred)
# dataset.decode_tam_word(y)